<a href="https://colab.research.google.com/github/edercarmonaa/IntroIA/blob/main/Sudoku.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install anytree


In [ ]:
import numpy as np
from anytree import Node, RenderTree
from collections import deque
import time
import resource

In [ ]:
def is_valid(board, num, row, col):
    # Verificar si el número se puede colocar en la posición (fila, columna)
    # sin violar las reglas del Sudoku.
    for i in range(9):
        if board[row][i] == num or board[i][col] == num:
            return False

    start_row, start_col = 3 * (row // 3), 3 * (col // 3)
    for i in range(3):
        for j in range(3):
            if board[start_row + i][start_col + j] == num:
                return False
    return True

def find_empty(board):
    # Encontrar la primera celda vacía del tablero.
    for i in range(9):
        for j in range(9):
            if board[i][j] == 0:
                return i, j
    return None, None


In [ ]:
def solve_sudoku_bfs(board):
    root = Node("Root", board=board)
    queue = deque([root])

    while queue:
        node = queue.popleft()
        empty_row, empty_col = find_empty(node.board)

        if empty_row is None or empty_col is None:
            # No hay celdas vacías, el tablero está resuelto.
            return node.board

        for num in range(1, 10):
            if is_valid(node.board, num, empty_row, empty_col):
                new_board = np.copy(node.board)
                new_board[empty_row][empty_col] = num
                child_node = Node("({}, {}) = {}".format(empty_row, empty_col, num), parent=node, board=new_board)
                queue.append(child_node)

In [ ]:
def solve_sudoku_dfs(board):
    root = Node("Root", board=board)
    stack = [root]

    while stack:
        node = stack.pop()
        empty_row, empty_col = find_empty(node.board)

        if empty_row is None or empty_col is None:
            # No hay celdas vacías, el tablero está resuelto.
            return node.board

        for num in range(1, 10):
            if is_valid(node.board, num, empty_row, empty_col):
                new_board = np.copy(node.board)
                new_board[empty_row][empty_col] = num
                child_node = Node("({}, {}) = {}".format(empty_row, empty_col, num), parent=node, board=new_board)
                stack.append(child_node)

In [ ]:
def solve_sudoku_iddfs(board):
    max_depth = 0
    root = Node("Root", board=board)

    while True:
        stack = [root]
        while stack:
            node = stack.pop()
            if node.depth == max_depth:
                continue

            empty_row, empty_col = find_empty(node.board)

            if empty_row is None or empty_col is None:
                # No hay celdas vacías, el tablero está resuelto.
                return node.board

            for num in range(1, 10):
                if is_valid(node.board, num, empty_row, empty_col):
                    new_board = np.copy(node.board)
                    new_board[empty_row][empty_col] = num
                    child_node = Node("({}, {}) = {}".format(empty_row, empty_col, num), parent=node, board=new_board)
                    stack.append(child_node)

        max_depth += 1

In [ ]:
# Ejemplo de tablero Sudoku (0 representa celdas vacías)
sudoku_board = np.array([
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
])

# Resolver el Sudoku utilizando BFS
start_time = time.time()
start_resources = resource.getrusage(resource.RUSAGE_SELF)
result_bfs = solve_sudoku_bfs(sudoku_board)
end_time = time.time()
end_resources = resource.getrusage(resource.RUSAGE_SELF)
execution_time_bfs = end_time - start_time
consumed_resources_bfs = end_resources.ru_maxrss - start_resources.ru_maxrss

print("Resolución usando BFS:")
print(result_bfs)
print(f"Tiempo de ejecución: {execution_time_bfs:.6f} segundos")
print(f"Consumo de recursos: {consumed_resources_bfs} KB")

# Resolver el Sudoku utilizando DFS
start_time = time.time()
start_resources = resource.getrusage(resource.RUSAGE_SELF)
result_dfs = solve_sudoku_dfs(sudoku_board)
end_time = time.time()
end_resources = resource.getrusage(resource.RUSAGE_SELF)
execution_time_dfs = end_time - start_time
consumed_resources_dfs = end_resources.ru_maxrss - start_resources.ru_maxrss

print("\nResolución usando DFS:")
print(result_dfs)
print(f"Tiempo de ejecución: {execution_time_dfs:.6f} segundos")
print(f"Consumo de recursos: {consumed_resources_dfs} KB")

# Resolver el Sudoku utilizando IDDFS
start_time = time.time()
start_resources = resource.getrusage(resource.RUSAGE_SELF)
result_iddfs = solve_sudoku_iddfs(sudoku_board)
end_time = time.time()
end_resources = resource.getrusage(resource.RUSAGE_SELF)
execution_time_iddfs = end_time - start_time
consumed_resources_iddfs = end_resources.ru_maxrss - start_resources.ru_maxrss

print("\nResolución usando IDDFS:")
print(result_iddfs)
print(f"Tiempo de ejecución: {execution_time_iddfs:.6f} segundos")
print(f"Consumo de recursos: {consumed_resources_iddfs} KB")

for pre, _, node in RenderTree(Node("Root", board=result_iddfs)):
    print(f"{pre}{node.name}")
